In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from transformers import AutoModel, AutoTokenizer
import torch
from torch import FloatTensor
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

c:\Miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Глобальные константы, прописываем в отдельные преременные для удобства

RANDOM_SEED = 42
PATH_TO_RAW_DATA = os.path.join('data', 'raw', 'geo-reviews-dataset-2023.csv')
NUM_LABLES = 6
BATCH_SIZE = 40

In [3]:
# Загружаем датасэт

raw_data_set = pd.read_csv(PATH_TO_RAW_DATA)
print(f'Объем сырого датасэта: {len(raw_data_set)}')
# Преобразуем метки классов для дальнейшей грамотной работы с CUDA
# raw_data_set['rating'] = raw_data_set['rating'] - 1
raw_data_set.head(5)

Объем сырого датасэта: 500000


,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [ ]:
lables = raw_data_set['rating'].unique() # возвращает объект типа np.ndarray
lables

In [4]:
# Бьем на train и test, defaul shuffle = True

train_X, test_X, train_Y, test_Y = train_test_split(raw_data_set['text'], raw_data_set['rating'], train_size=0.3, test_size=0.1, random_state=RANDOM_SEED, stratify=raw_data_set['rating'])

In [5]:
# Токенизируем тексты
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny')
train_tokens = tokenizer(list(train_X), padding = True, truncation=True)
test_tokens = tokenizer(list(test_X), padding = True, truncation=True)

print(train_tokens.keys())
print(train_tokens['input_ids'][0])
print(tokenizer.decode(train_tokens['input_ids'][0]))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[101, 468, 16948, 20442, 16948, 28410, 19692, 19692, 487, 19692, 28403, 28404, 16948, 498, 17424, 28403, 28404, 10286, 14800, 477, 16948, 28396, 10286, 117, 488, 19692, 490, 28400, 16948, 28407, 16948, 17106, 490, 28400, 14800, 28397, 119, 450, 477, 28413, 28407, 16948, 28396, 17127, 28413, 19692, 477, 19692, 20442, 16948, 14800, 28404, 17127, 16948, 487, 17127, 16948, 28395, 16948, 488, 10286, 20442, 16948, 28396, 28405, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
class CustomDataSet(Dataset):
    def __init__(self,
                 tokenized_text,
                 attention_mask,
                 lables):
        self.tokenized_text = torch.tensor(tokenized_text)
        self.attention_mask = torch.tensor(attention_mask)
        self.lables = torch.tensor(lables.to_numpy())

    def __len__(self):
        return len(self.lables)

    def __getitem__(self, idx):
        return self.tokenized_text[idx], self.attention_mask[idx], self.lables[idx]

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
batch_size = 40
train_dataset = CustomDataSet(train_tokens['input_ids'], train_tokens['attention_mask'], train_Y)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = CustomDataSet(test_tokens['input_ids'], test_tokens['attention_mask'], test_Y)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=NUM_LABLES)

# model = AutoModel.from_pretrained("cointegrated/rubert-tiny") # Pre-trained model
optimizer = AdamW(model.parameters(), lr=1e-5) # Optimization function
loss_fn = torch.nn.CrossEntropyLoss() # Loss function

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
num_epochs = 3
model.to(device) # Transfer model to GPU if available

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
import tqdm

train_losses = []
val_losses = []

train_losses_per_epoch = []
val_losses_per_epoch = []

for epoch in tqdm.tqdm(range(num_epochs)):
    # TRAINING BLOCK STARTS
    model.train()
    for batch in train_loader:

        tokenized_text, attention_mask, lables = batch
        lables = lables.type(torch.LongTensor)
        tokenized_text, attention_mask, lables = tokenized_text.to(device), attention_mask.to(device), lables.to(device)
        
        # Setting the gradients to zero
        optimizer.zero_grad()
        
        # Passing the data to the model
        outputs = model(input_ids = tokenized_text, attention_mask = attention_mask)
        
        # The logits will be used for measuring the loss
        pred = outputs.logits
        loss = loss_fn(pred, lables)

        # Calculating the gradient for the loss function
        loss.backward()
        
        # Optimizing the parameters of the bert model
        optimizer.step()

        # Calculating the running loss for logging purposes
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))
    # TRAINING BLOCK ENDS 

    # TESTING BLOCK STARTS
    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            tokenized_text, attention_mask, lables = batch
            lables = lables.type(torch.LongTensor)
            tokenized_text, attention_mask, lables = tokenized_text.to(device), attention_mask.to(device), lables.to(device)
            
            # We don't need gradients for testing
            outputs = model(input_ids = tokenized_text, attention_mask = attention_mask)
            
            # Logits act as predictions
            logits = outputs.logits
            
            # Calculating total batch loss using the logits and labels
            loss = loss_fn(logits, lables)
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))
    val_losses_per_epoch = []
    train_losses_per_epoch = []

  0%|          | 0/3 [00:00<?, ?it/s]